In [ ]:
'''匯入套件'''
import requests as req
from bs4 import BeautifulSoup as bs
import json, os, time, random, pprint, re

# 隨機取得 User-Agent
'''
# 從外部資料來取得清單，清單預設儲存路徑: /tmp
ua = UserAgent(use_external_data=True)
# 從外部資料來取得清單，儲存在指定路徑
ua = UserAgent(use_external_data=True, cache_path=/home/fake_useragent.json)

更詳細的說明，請見以下網頁:
https://pypi.org/project/fake-useragent/
'''
from fake_useragent import UserAgent
ua = UserAgent(use_external_data=True)

'''放置 金庸小說 metadata 的資訊'''
listData = []

'''金庸小說的網址'''
prefix = 'https://hetubook.com'
list_urls = [
    prefix + '/tag/金庸-1.html',
    prefix + '/tag/金庸-2.html'
]

'''設定標頭'''
my_headers = {
    'user-agent': ua.random,
    'referer': 'https://hetubook.com/book2/56/index.html',
}


# 沒有放置 txt 檔的資料夾，就建立起來
folderPath = 'jinyong'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

In [ ]:
'''
自訂函式
'''
# 取得小說的主要連結
def getMainLinks():    
    # 清除 list 內容
    listData.clear()
    
    # 走訪首頁
    for link in list_urls:
        res = req.get(url = link, headers = my_headers, cookies = my_cookies)
        soup = bs(res.text, "lxml")

        # 取得每一本指定的小說連結
        for a in soup.select('dl#body.list dd h4 a[href]'):
            # 取得主要連結相關資訊
            listData.append({
                'title': a.get_text(),
                'link': prefix + a['href'],
                'sub': [] # 之後會放置每一本小說的章回資訊
            })
        
    # 預覽結果
    # pprint.pprint(listData)
    

# 取得所有小說的獨立連結
def getSubLinks():
    # 取得章回列表
    for index in range( len(listData) ):
        res = req.get(url = listData[index]['link'], headers = my_headers)
        soup = bs(res.text, "lxml")
        
        for a in soup.select('dl#dir > dd > a[href][title]'):
            listData[index]['sub'].append({
                'title': a['title'], # 或是 a.get_text() 來取得 title
                'link': prefix + a['href'],
                'content': '' # 預留給小說內文
            })
            
        # 隨機等待
        # time.sleep(random.randint(1, 3))
    
    # 預覽結果
    # pprint.pprint(listData)
    
    
# 將金庸小說所有章回的內容，各自寫到 txt 與 json 中
def writeTxt():
    for index in range( len(listData) ):
        for idx in range( len(listData[index]['sub']) ):
            sess = req.Session()
            res = sess.get(url = listData[index]['sub'][idx]['link'], headers = my_headers)
            for key, value in res.cookies.items():
                print(key + '=' + value)
            
            soup = bs(res.text, "lxml")
            
            # 用 len( soup.select('CSS_SELECTOR') ) 來判斷文章區域是否存在，存在則儲存起來
            if len(soup.select('div#content')) > 0:
                # 取得內文所在的元素
                elm = soup.select_one('div#content')
                
                # 刪除不必要的元素
                elm.select_one('h2.h2').decompose()
                
                # 取得小說內文
                content = ''
                for div in elm.select('div'):
                    content += div.get_text()
                
                pprint.pprint(elm)
                
                # 更新 json 的 content 節點
                listData[index]['sub'][idx]['content'] = content
                
                # 將小說內文額外存成 txt 檔
                file_name = f"{listData[index]['title']}_{listData[index]['sub'][idx]['title']}"
                with open(f"{folderPath}/{file_name}.txt", "w", encoding="utf-8") as file:
                    file.write(content)
                    
            break
                    
            # 隨機等待
            # time.sleep(random.randint(1, 3))
            
        break
                

# 建立金庸小說的 json 檔
def saveJson():
    with open(f"{folderPath}/hetubook_jinyong_requests.json", "w", encoding="utf-8") as file:
        file.write( json.dumps(listData, ensure_ascii=False, indent=4) )

In [ ]:
# 主程式
if __name__ == "__main__":
    time_begin = time.time()
    getMainLinks()
    getSubLinks()
    writeTxt()
#     saveJson()
    time_end = time.time()
    print(f"總共執行 {time_end - time_begin} 秒")